# シンプレックス法(単体法)

\begin{eqnarray}
&maximize& \ \ f(x) = cx \\
&subject \ to&\ Ax \leq b 
\end{eqnarray}


### 問題1
$$
\begin{align}
maximize& \quad z = \left( \begin{array}{cc} 3 & 2 \end{array} \right) 
\left( \begin{array}{c} x_1 \\ x_2  \end{array} \right) \\
subject \ \  to & \quad  
\left( \begin{array}{cc} 3 & 1 \\ 2.5 & 2 \\ 1 & 2 \end{array}  \right) 
\left( \begin{array}{c} x_1 \\ x_2  \end{array} \right)
\leq \left( \begin{array}{c} 9 \\ 12.5 \\ 8  \end{array} \right) , \quad x_1 ,\ x_2  \geq 0
\end{align}
$$

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
c = np.array( [3,2] )
A = np.array([ [3,1],[2.5,2],[1,2]])
comp = np.array([-1,-1,-1])
b = np.array([9,12.5,8])

n,m = A.shape
slackVariableNum = 0
artificialVariableNum = 0

slackVariable = [0] * n
artificialVariable = [0] * n
for i in range(n):
    # bの値を全て正の値にしておく
    if b[i] < 0:
        A[i] = -A[i]
        comp[i] = -comp[i]
        b[i] = -b[i]
    # < ( -> スラック変数を導入 )
    if comp[i] == -1:
        slackVariableNum += 1
        slackVariable[i] = 1
    # = ( -> 人為変数を導入 )
    elif comp[i] == 0:
        artificialVariableNum += 1
        artificialVariable[i] = 1
    # > ( -> スラック変数,人為変数を導入 )
    else:
        slackVariableNum += 1
        artificialVariableNum += 1
        slackVariable[i] = -1
        artificialVariable[i] = 1

variableNum = c.shape[0] + slackVariableNum + artificialVariableNum
addVariableNum =  slackVariableNum + artificialVariableNum

# 基底可能解を求める.
baseIndex = np.empty(n)
baseValue = np.empty(n)
A_ = np.append(A , np.zeros((n,addVariableNum)),axis=1)
slackIter = c.shape[0] 
artificialIter = c.shape[0] + slackVariableNum

# (スラック変数 < 人為変数) の優先順位で基底変数に選ぶ.
# すると , i 本目の制約条件式のみに登場する変数を選ぶことができる.
# baseIndex[i] := i 本目の制約条件式のみに登場する変数の番号
# baseValue[i] := i本目の制約条件式のみに登場する変数の値 ( = 基底可能解 = b[i] ) となる.
for i in range(n):
    if slackVariable[i] != 0:
        A_[i,slackIter] = slackVariable[i]
        # 1の場合
        if slackVariable[i] > 0:
            baseIndex[i],baseValue[i] = slackIter, b[i]
        slackIter += 1
            
    if artificialVariable[i] != 0:
        A_[i,artificialIter] = artificialVariable[i]
        baseIndex[i],baseValue[i] = artificialIter, b[i]
        artificialIter += 1

print(baseIndex)
print(baseValue)

[ 2.  3.  4.]
[  9.   12.5   8. ]


In [3]:
# フェーズ1 (基底可能解を見つける)
# 目的関数の値をzとおく
# 基底可能解の列を追加
exA = np.append(baseValue.reshape(n,1),A_,axis=1)
# zの行を追加
c_ = np.array([0]*(c.shape[0] + slackVariableNum) + [-1]*(artificialVariableNum))
c_ = c_[np.vectorize(int)(baseIndex)]
w = (c_ @ exA).reshape(1,variableNum+1)
z = np.append(np.append(np.zeros(1),-c),np.array([0]*addVariableNum)).reshape(1,variableNum+1)
table = np.append(np.append(exA,w,axis=0),z,axis=0)
# データフレームにする
df = pd.DataFrame(table,
                columns=['基底可能解']+[ 'x' + str(i) for i in range(variableNum)],
                index= list(np.vectorize(lambda i: 'x' + str(int(i)))(baseIndex))  + ['w','z']
        )
df

,基底可能解,x0,x1,x2,x3,x4
x2,9.0,3.0,1.0,1.0,0.0,0.0
x3,12.5,2.5,2.0,0.0,1.0,0.0
x4,8.0,1.0,2.0,0.0,0.0,1.0
w,0.0,0.0,0.0,0.0,0.0,0.0
z,0.0,-3.0,-2.0,0.0,0.0,0.0


In [4]:
# 単体表をもらってからの処理
def optimize(table,target):
    baseIndex = table.index.values
    nonBaseIndex = np.setdiff1d(np.vectorize(lambda i : 'x' + str(i))(np.arange(len(table.columns)-1)) ,baseIndex)
    for _ in range(5):
        display(table)
        nonBaseTable = table.loc[target,nonBaseIndex]
        if ((nonBaseTable < -1e-8).values.sum()) == 0:
            return table
        # 新たな基底変数
        nextIndex = (nonBaseTable.map(lambda x: -x)).idxmax(axis=1)
        # 取り替えられる基底変数
        idx = table.index.get_loc(target)
        tmpLine = (table['基底可能解'].iloc[:idx] / table.loc[ : ,nextIndex].iloc[:idx] )
        prevIndex = str(tmpLine.map(lambda x: float('inf') if x < 0 else x ).idxmin())
        nonBaseIndex[np.where(nonBaseIndex == nextIndex)] = prevIndex
        table = table.rename(index={prevIndex : nextIndex})
        table.loc[nextIndex] /= table.at[nextIndex,nextIndex]
        pivotLine = table.loc[nextIndex]
        unPivotIndex = list(table.index.drop(nextIndex))
        table.loc[unPivotIndex] = table.loc[unPivotIndex].apply(lambda x: x - (x.at[nextIndex]*pivotLine) ,axis=1)

table = optimize(df,'w')
display(table)
if abs(table['基底可能解']['w']) > 1e-8:
    print("No Answer")
    exit(1)

,基底可能解,x0,x1,x2,x3,x4
x2,9.0,3.0,1.0,1.0,0.0,0.0
x3,12.5,2.5,2.0,0.0,1.0,0.0
x4,8.0,1.0,2.0,0.0,0.0,1.0
w,0.0,0.0,0.0,0.0,0.0,0.0
z,0.0,-3.0,-2.0,0.0,0.0,0.0


,基底可能解,x0,x1,x2,x3,x4
x2,9.0,3.0,1.0,1.0,0.0,0.0
x3,12.5,2.5,2.0,0.0,1.0,0.0
x4,8.0,1.0,2.0,0.0,0.0,1.0
w,0.0,0.0,0.0,0.0,0.0,0.0
z,0.0,-3.0,-2.0,0.0,0.0,0.0


In [5]:
#　フェーズ2(解く)
if artificialVariableNum != 0:
    table = table.iloc[:,:-artificialVariableNum]
variableNum -= artificialVariableNum
table = table.drop('w')
display(table)

,基底可能解,x0,x1,x2,x3,x4
x2,9.0,3.0,1.0,1.0,0.0,0.0
x3,12.5,2.5,2.0,0.0,1.0,0.0
x4,8.0,1.0,2.0,0.0,0.0,1.0
z,0.0,-3.0,-2.0,0.0,0.0,0.0


In [7]:
result = optimize(table,'z')
result

,基底可能解,x0,x1,x2,x3,x4
x2,9.0,3.0,1.0,1.0,0.0,0.0
x3,12.5,2.5,2.0,0.0,1.0,0.0
x4,8.0,1.0,2.0,0.0,0.0,1.0
z,0.0,-3.0,-2.0,0.0,0.0,0.0


,基底可能解,x0,x1,x2,x3,x4
x0,3.0,1.0,0.333333,0.333333,0.0,0.0
x3,5.0,0.0,1.166667,-0.833333,1.0,0.0
x4,5.0,0.0,1.666667,-0.333333,0.0,1.0
z,9.0,0.0,-1.000000,1.000000,0.0,0.0


,基底可能解,x0,x1,x2,x3,x4
x0,2.0,1.0,0.0,0.4,0.0,-0.2
x3,1.5,0.0,0.0,-0.6,1.0,-0.7
x1,3.0,0.0,1.0,-0.2,0.0,0.6
z,12.0,0.0,0.0,0.8,0.0,0.6


,基底可能解,x0,x1,x2,x3,x4
x0,2.0,1.0,0.0,0.4,0.0,-0.2
x3,1.5,0.0,0.0,-0.6,1.0,-0.7
x1,3.0,0.0,1.0,-0.2,0.0,0.6
z,12.0,0.0,0.0,0.8,0.0,0.6
